In [8]:
# tag preproces
import os

tags_file = "../data/img_tags.csv"
tagsmap = {
    0: ['风光'],
    1: ['人文'],
    2: ['人像', '儿童'],
    3: ['其他', '生态', '活动', '体育', '美食', '小品']
}
img2tag = {}
tag_count = {}
with open(tags_file, 'r') as fin:
    for line in fin:
        fields = line.strip().split("\t")
        if len(fields) != 2:
            continue
        imgid, tag = fields
        for k, v in tagsmap.items():
            if tag in v:
                img2tag[int(imgid)] = k
                tag_count.setdefault(k, 0)
                tag_count[k] += 1
        
for tag, count in sorted(tag_count.items(), key=lambda x: x[0]):
    print tag, count

0 16346
1 5062
2 3259
3 4674


In [33]:
# model define
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalMaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.utils import to_categorical
from keras import optimizers
import numpy as np

model = Sequential([
    GlobalMaxPool2D(input_shape=(2048, 8, 8)),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax')
    ])

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

In [ ]:
# load train and valid data
# need to run only once
feature_path = "../data/pretrained_features/"
data_path = "./data/"

for npz in os.listdir(feature_path):
    print "loading npzfile: %s"%npz

    X = []
    Y = []
    npzfile = np.load(feature_path+npz)
    img_ids = npzfile['img_ids']
    img_features = npzfile['img_features']
    for _ in range(len(img_ids)):
        imgid= img_ids[_]
        feature = img_features[_]
        if imgid in img2tag:
            X.append(feature)
            Y.append(img2tag[imgid])

    X = np.array(X)
    Y = to_categorical(np.array(Y), num_classes=4)
    
    np.savez(data_path+npz, X = X, Y = Y)

In [34]:
# train model
import random

data_path = "./data/"
npzfile_list = os.listdir(data_path)

npz = np.load(data_path+"imgs_epoch=25.npz")
X_valid = npz['X']
Y_valid = npz['Y']

for epoch in range(10):
    random.shuffle(npzfile_list)
    for npzfile in npzfile_list:
        if npzfile == "imgs_epoch=25.npz":
            continue
        print "loading npzfile: %s"%npzfile

        npz = np.load(data_path+npzfile)
        X = npz['X']
        Y = npz['Y']
        
        model.fit(X, Y, batch_size=64, epochs=1, validation_data=(X_valid, Y_valid), shuffle=True,
                  class_weight={0:0.2, 1:0.64, 2:1.0, 3:0.70})

loading npzfile: imgs_epoch=8.npz
Train on 1160 samples, validate on 90 samples
Epoch 1/1
1160/1160 [==============================] - 0s - loss: 1.0299 - acc: 0.4397 - val_loss: 8.7526 - val_acc: 0.1444
loading npzfile: imgs_epoch=3.npz
Train on 1177 samples, validate on 90 samples
Epoch 1/1
1177/1177 [==============================] - 0s - loss: 0.7514 - acc: 0.5157 - val_loss: 3.9642 - val_acc: 0.5889
loading npzfile: imgs_epoch=19.npz
Train on 1168 samples, validate on 90 samples
Epoch 1/1
1168/1168 [==============================] - 0s - loss: 0.7115 - acc: 0.5068 - val_loss: 10.2333 - val_acc: 0.1889
loading npzfile: imgs_epoch=14.npz
Train on 1207 samples, validate on 90 samples
Epoch 1/1
1207/1207 [==============================] - 0s - loss: 0.6737 - acc: 0.4996 - val_loss: 3.5973 - val_acc: 0.4778
loading npzfile: imgs_epoch=17.npz
Train on 1139 samples, validate on 90 samples
Epoch 1/1
1139/1139 [==============================] - 0s - loss: 0.6639 - acc: 0.5391 - val_loss: 2

Train on 1186 samples, validate on 90 samples
Epoch 1/1
1186/1186 [==============================] - 0s - loss: 0.4679 - acc: 0.5911 - val_loss: 0.7451 - val_acc: 0.7111
loading npzfile: imgs_epoch=0.npz
Train on 1139 samples, validate on 90 samples
Epoch 1/1
1139/1139 [==============================] - 0s - loss: 0.4650 - acc: 0.5891 - val_loss: 0.7326 - val_acc: 0.7111
loading npzfile: imgs_epoch=24.npz
Train on 1184 samples, validate on 90 samples
Epoch 1/1
1184/1184 [==============================] - 0s - loss: 0.3905 - acc: 0.6208 - val_loss: 0.8247 - val_acc: 0.6444
loading npzfile: imgs_epoch=2.npz
Train on 1154 samples, validate on 90 samples
Epoch 1/1
1154/1154 [==============================] - 0s - loss: 0.4245 - acc: 0.6404 - val_loss: 0.8071 - val_acc: 0.6111
loading npzfile: imgs_epoch=22.npz
Train on 1176 samples, validate on 90 samples
Epoch 1/1
1176/1176 [==============================] - 0s - loss: 0.4011 - acc: 0.6207 - val_loss: 0.8768 - val_acc: 0.6333
loading npzf

1160/1160 [==============================] - 0s - loss: 0.3648 - acc: 0.6724 - val_loss: 0.8356 - val_acc: 0.6000
loading npzfile: imgs_epoch=0.npz
Train on 1139 samples, validate on 90 samples
Epoch 1/1
1139/1139 [==============================] - 0s - loss: 0.4249 - acc: 0.6076 - val_loss: 0.7995 - val_acc: 0.5889
loading npzfile: imgs_epoch=14.npz
Train on 1207 samples, validate on 90 samples
Epoch 1/1
1207/1207 [==============================] - 0s - loss: 0.3910 - acc: 0.6131 - val_loss: 0.7312 - val_acc: 0.6778
loading npzfile: imgs_epoch=16.npz
Train on 1159 samples, validate on 90 samples
Epoch 1/1
1159/1159 [==============================] - 0s - loss: 0.3960 - acc: 0.6152 - val_loss: 0.7952 - val_acc: 0.6333
loading npzfile: imgs_epoch=7.npz
Train on 1160 samples, validate on 90 samples
Epoch 1/1
1160/1160 [==============================] - 0s - loss: 0.3603 - acc: 0.6466 - val_loss: 0.8277 - val_acc: 0.6111
loading npzfile: imgs_epoch=6.npz
Train on 1162 samples, validate on

1134/1134 [==============================] - 0s - loss: 0.3782 - acc: 0.6429 - val_loss: 0.9392 - val_acc: 0.5889
loading npzfile: imgs_epoch=21.npz
Train on 1147 samples, validate on 90 samples
Epoch 1/1
1147/1147 [==============================] - 0s - loss: 0.3597 - acc: 0.6530 - val_loss: 0.8852 - val_acc: 0.5556
loading npzfile: imgs_epoch=6.npz
Train on 1162 samples, validate on 90 samples
Epoch 1/1
1162/1162 [==============================] - 0s - loss: 0.3998 - acc: 0.6222 - val_loss: 0.8274 - val_acc: 0.6444
loading npzfile: imgs_epoch=4.npz
Train on 1186 samples, validate on 90 samples
Epoch 1/1
1186/1186 [==============================] - 0s - loss: 0.4282 - acc: 0.6113 - val_loss: 0.8650 - val_acc: 0.6222
loading npzfile: imgs_epoch=14.npz
Train on 1207 samples, validate on 90 samples
Epoch 1/1
1207/1207 [==============================] - 0s - loss: 0.3805 - acc: 0.5982 - val_loss: 0.7960 - val_acc: 0.6222
loading npzfile: imgs_epoch=8.npz
Train on 1160 samples, validate on

KeyboardInterrupt: 

In [ ]:
# save model
model.save("./model.h5")

In [2]:
16346 + 5062 + 3259 + 4674

29341

In [6]:
3259.0 / 4674

0.6972614462986735

In [30]:
np.sum(Y_valid,axis=0)

array([ 53.,  16.,   9.,  12.])